In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

from data_processing import *

**Get the paths to olfactory bulb data**

In [ ]:

path_root = r"E:\place_decoding\data"
ob_dir = 'bulb'

# Paths to olfactory bulb data:
ob_paths = []
ob_paths_sniffing = []
mouse_ids = os.listdir(os.path.join(path_root, ob_dir))
for mouse_id in mouse_ids:
    session_ids = os.listdir(os.path.join(path_root, ob_dir, mouse_id))
    for session_id in session_ids:
        session_path = os.path.join(path_root, ob_dir, mouse_id, session_id)
        files = os.listdir(session_path)
        if 'track.mat' or 'mtracks.mat' in files:
            ob_paths.append(session_path)
        if 'track.mat' or 'mtracks.mat' in files and 'sniff_params' in files:
            ob_paths_sniffing.append(session_path)

print('\nNumber of OB sessions with tracking data: ', len(ob_paths))
print('\nNumber of OB sessions with tracking and sniffing data: ', len(ob_paths_sniffing))






**Get the paths to hippocampal data**

In [2]:
path_root = r"E:\place_decoding\data"
hipp_dir = 'hipp'
            
            
# Paths to hippocampus data:
hipp_paths = []
hipp_paths_sniffing = []
mouse_ids = os.listdir(os.path.join(path_root, hipp_dir))
for mouse_id in mouse_ids:
    session_ids = os.listdir(os.path.join(path_root, hipp_dir, mouse_id))
    for session_id in session_ids:
        session_path = os.path.join(path_root, hipp_dir, mouse_id, session_id)
        files = os.listdir(session_path)
        if 'track.mat' or 'mtracks.mat' in files:
            hipp_paths.append(session_path)
        if 'track.mat' or 'mtracks.mat' in files and 'sniff_params' in files:
            hipp_paths_sniffing.append(session_path)

print('\nNumber of hippocampal sessions with tracking data: ', len(hipp_paths))
print('\nNumber of hippocampal sessions with tracking and sniffing data: ', len(hipp_paths_sniffing))



Number of hippocampal sessions with tracking data:  16

Number of hippocampal sessions with tracking and sniffing data:  16


***Make plots for all of the olfactory bulb sessions with tracking data.***

We ignore the floor flip for now.

In [ ]:
# Make plots for all of the olfactory bulb sessions with tracking data. We ignore the floor flip for now.
for path in ob_paths:
    print('Analyzing ' + path)

    _ = preprocess_data(path, bin_size=20, make_plots=True)
    


***Make plots for all of the hippocampal sessions with tracking data.***

In [ ]:

for path in hipp_paths:
    print('Analyzing ' + path)

    _ = preprocess_data(path, bin_size=20, make_plots=True)
    


In [3]:
# Number of boxes along each dimension for decoding:
nx, ny = 12, 5

# The box boundaries for spatial tracking (see readme file):
x_min, x_max = 0, 1200
y_min, y_max = 0, 500

# The minimum and maximum error that we would expect given the size of the bins:
dx = (x_max-x_min)/nx
dy = (y_max-y_min)/ny
err_min = ((dx**2 + dy**2)/12)**0.5
err_chance = (((x_max-x_min)**2 + (y_max-y_min)**2)/12)**0.5
print('Minimum achievable error: ', err_min)
print('Chance performance error: ', err_chance)

Minimum achievable error:  40.824829046386306
Chance performance error:  375.27767497325675


In [4]:

[hipp_paths]

[['E:\\place_decoding\\data\\hipp\\4132\\3',
  'E:\\place_decoding\\data\\hipp\\4132\\4',
  'E:\\place_decoding\\data\\hipp\\4132\\5',
  'E:\\place_decoding\\data\\hipp\\4132\\6',
  'E:\\place_decoding\\data\\hipp\\4133\\1',
  'E:\\place_decoding\\data\\hipp\\4133\\2',
  'E:\\place_decoding\\data\\hipp\\4133\\4',
  'E:\\place_decoding\\data\\hipp\\4133\\5',
  'E:\\place_decoding\\data\\hipp\\4134\\2',
  'E:\\place_decoding\\data\\hipp\\4134\\3',
  'E:\\place_decoding\\data\\hipp\\4134\\4',
  'E:\\place_decoding\\data\\hipp\\4135\\1',
  'E:\\place_decoding\\data\\hipp\\4135\\2',
  'E:\\place_decoding\\data\\hipp\\4137\\1',
  'E:\\place_decoding\\data\\hipp\\4137\\2',
  'E:\\place_decoding\\data\\hipp\\4137\\3']]

### Spatial Decoding

***decoding all the olfactor bulb sessions without considering the floor flip***

In [ ]:
# Do the spatial decoding with no floor flip for the olfactory bulb data: 


bin_size = 20  # number of 10ms time steps to include in each bin

for path in ob_paths:
    print('Analyzing ' + path)

    # preprocess the data getting the spike counts and position:
    spks, pos, _, _ = preprocess_data(path, bin_size=bin_size)
    if spks is None:
        continue
    

    # calculate the decoding error on the 2D grid:
    err, _, pos_pred = decoding_err_multithreadCV(pos, spks, n_squares=(nx,ny), 
                            boundaries=(x_min,x_max,y_min,y_max))
    np.save(path + '/decoding_err.npy', err)
    np.save(path + '/pos_pred.npy', pos_pred)


    # calculate the decoding error with shuffling:
    err_shuffle, _, pos_pred_shuffle = decoding_err_multithreadCV(pos, spks, n_squares=(nx,ny), 
                        boundaries=(x_min,x_max,y_min,y_max), shuffle=True)
    np.save(path + '/decoding_err_shuffle.npy', err_shuffle)
    np.save(path + '/pos_pred_shuffle.npy', pos_pred_shuffle)
    

***Decoding all the hippocampal sessions without considering the floor flip***

In [ ]:

# Do the spatial decoding with no floor flip for the hippocampal data: 


bin_size = 20  # number of 10ms time steps to include in each bin

for path in hipp_paths:
    print('Analyzing ' + path)

    # preprocess the data getting the spike counts and position:
    spks, pos, _, _ = preprocess_data(path, bin_size=bin_size)
    if spks is None:
        continue
    

    # calculate the decoding error on the 2D grid:
    err, _, pos_pred = decoding_err_multithreadCV(pos, spks, n_squares=(nx,ny), 
                            boundaries=(x_min,x_max,y_min,y_max))
    np.save(path + '/decoding_err.npy', err)
    np.save(path + '/pos_pred.npy', pos_pred)


    # calculate the decoding error with shuffling:
    err_shuffle, _, pos_pred_shuffle = decoding_err_multithreadCV(pos, spks, n_squares=(nx,ny), 
                        boundaries=(x_min,x_max,y_min,y_max), shuffle=True)
    np.save(path + '/decoding_err_shuffle.npy', err_shuffle)
    np.save(path + '/pos_pred_shuffle.npy', pos_pred_shuffle)

Analyzing E:\place_decoding\data\hipp\4132\3
n_neurons:  39
Spatial decoding error:  303.85406013656586 +/- 42.72103020391384
n_neurons:  39
Spatial decoding error:  288.38272565104927 +/- 42.06961172151979
Analyzing E:\place_decoding\data\hipp\4132\4
n_neurons:  90
Spatial decoding error:  241.77316882351755 +/- 23.53583081790263
n_neurons:  90
Spatial decoding error:  302.84896612615046 +/- 31.29305382042325
Analyzing E:\place_decoding\data\hipp\4132\5
n_neurons:  57


***Decoding all the olfactory bulb sessions considering the floor flip***

In [ ]:
# Do the spatial decoding for the olfactory bulb sessions, with floor flip

bin_size = 20  # number of 10ms time steps to include in each bin

# Define training and testing conditions and shuffle states
train_test_conditions = [
    ('pre', 'pre'),
    ('post', 'post'),
    ('pre', 'post'),
    ('post', 'pre')
]
shuffle_states = [False, True]

for path in ob_paths:  # Loop through all sessions
    if 'events.mat' in os.listdir(path):
        events = loadmat(path + '/events.mat')['events']
        frame_flip1 = events[2, 0]  # Frame at which floor flip begins

        if frame_flip1 != 0:  # Only analyze if there's a floor flip

            # Preprocess the data
            spks_pre, pos_pre, _, _ = preprocess_data(path, bin_size=bin_size, flip_data='pre')
            spks_post, pos_post, _, _ = preprocess_data(path, bin_size=bin_size, flip_data='post')

            for train_on, test_on in train_test_conditions:
                for shuffle in shuffle_states:
                    shuffle_str = '_shuffle' if shuffle else ''
                    train_test_str = f"train_{train_on}_test_{test_on}{shuffle_str}"

                    print(f'{train_test_str.capitalize()}...')

                    err, ent, pos_pred = decoding_err_floorflip(
                        pos_pre, pos_post, spks_pre, spks_post, 
                        n_squares=(nx, ny), boundaries=(x_min, x_max, y_min, y_max), 
                        kCV=10, shuffle=shuffle, train_on=train_on, test_on=test_on
                    )
                    
                    # Save results
                    np.save(path + f'/decoding_err_{train_test_str}.npy', err)
                    np.save(path + f'/pos_pred_{train_test_str}.npy', pos_pred)

        else:
            print('No floor flip, skipping session...')
            continue

    else:
        print('No events file, skipping session...')
        continue


***Decoding all the hippocampal sessions considering the floor flip***

In [ ]:
bin_size = 20  # number of 10ms time steps to include in each bin

# Define training and testing conditions and shuffle states
train_test_conditions = [
    ('pre', 'pre'),
    ('post', 'post'),
    ('pre', 'post'),
    ('post', 'pre')
]
shuffle_states = [False, True]

for path in hipp_paths:  # Loop through all sessions
    if 'events.mat' in os.listdir(path):
        events = loadmat(path + '/events.mat')['events']
        frame_flip1 = events[2, 0]  # Frame at which floor flip begins

        if frame_flip1 != 0:  # Only analyze if there's a floor flip

            # Preprocess the data
            spks_pre, pos_pre, _, _ = preprocess_data(path, bin_size=bin_size, flip_data='pre')
            spks_post, pos_post, _, _ = preprocess_data(path, bin_size=bin_size, flip_data='post')

            for train_on, test_on in train_test_conditions:
                for shuffle in shuffle_states:
                    shuffle_str = '_shuffle' if shuffle else ''
                    train_test_str = f"train_{train_on}_test_{test_on}{shuffle_str}"

                    print(f'{train_test_str.capitalize()}...')

                    err, ent, pos_pred = decoding_err_floorflip(
                        pos_pre, pos_post, spks_pre, spks_post, 
                        n_squares=(nx, ny), boundaries=(x_min, x_max, y_min, y_max), 
                        kCV=10, shuffle=shuffle, train_on=train_on, test_on=test_on
                    )
                    
                    # Save results
                    np.save(path + f'/decoding_err_{train_test_str}.npy', err)
                    np.save(path + f'/pos_pred_{train_test_str}.npy', pos_pred)

        else:
            print('No floor flip, skipping session...')
            continue

    else:
        print('No events file, skipping session...')
        continue
